In [19]:
import numpy as np
from numpy.linalg import inv, det
from scipy.stats import multivariate_normal
import math as m
from scipy.optimize import fmin_l_bfgs_b

import pandas as pd

from mpl_toolkits.mplot3d import Axes3D 
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_boston, load_diabetes
from sklearn.metrics import mean_squared_error

In [8]:
data = load_diabetes()

X = data.data
y = data.target

X_train = X[:400]
y_train = y[:400]
X_test = X[400:]
y_test = y[400:]

In [9]:
X.shape

(442, 10)

In [2]:
# data = load_boston()

# X = data.data
# y = data.target

# X_train = X[:450]
# y_train = y[:450]
# X_test = X[450:]
# y_test = y[450:]

In [10]:
X_mean = X_train.mean(axis=0)
X_std = X_train.std(axis=0)
y_mean = y_train.mean()
y_std = y_train.std()

In [11]:
X_train = X_train - X_mean
X_train = X_train / X_std

X_test = X_test - X_mean
X_test = X_test / X_std

y_train = y_train - y_mean
y_train = y_train / y_std

In [12]:
X_train = X_train + 100 * np.random.randn(X_train.shape[0], X_train.shape[1])

In [12]:
det(X_train.dot(X_train.T))

0.0

In [13]:
X_test.shape

(42, 10)

In [14]:
class GaussianProcessRegressionBFGS:
    def __init__(self):
        self.tau_0 = m.log(1.0)
        self.sigma_0 = m.log(1.0)
        self.eta_0 = m.log(1.0)
        self.theta_0 = np.array([self.tau_0, self.sigma_0, self.eta_0])
        
    def __Kernel(self, theta, x):
        theta1 = m.exp(theta[0])
        theta2 = m.exp(-theta[1])
        theta3 = m.exp(theta[2])

        kernel_matrix = np.zeros((x.shape[0], x.shape[0]))
        L2norm = np.zeros((x.shape[0], x.shape[0]))
        for i in range(x.shape[0]):
            for j in range(x.shape[0]):
                L2norm[i, j] = np.dot(x[i] - x[j], x[i] - x[j])
                kernel_matrix[i, j] = theta1 * m.exp(- L2norm[i, j]* theta2)
            
        kernel_matrix = kernel_matrix + theta3 * np.eye(x.shape[0]) 
        return kernel_matrix, L2norm
        
    def __logL(self, theta, *args):
        print(theta)
        x = args[0]
        y = args[1]
        
        K, _ = self.__Kernel(theta, x)
        print(det(K))
        loglik1 = - m.log(det(K))
        loglik2 = - y.reshape(-1,1).T.dot(inv(K)).dot(y.reshape(-1, 1))
        
        print(-(loglik1 + loglik2))
        
        return -(loglik1 + loglik2)
    
    def __dLdtheta(self, theta, *args):
        x = args[0]
        y = args[1]
        
        K, L2norm = self.__Kernel(theta, x)
        K_inv = inv(K)
        
        dK_tau = K - m.exp(theta[2]) * np.eye(x.shape[0])
        dL1_tau = -np.trace(K_inv.dot(dK_tau))
        Kinvy = np.dot(K_inv, y.reshape(-1, 1)) 
        dL2_tau = Kinvy.T.dot(dK_tau).dot(Kinvy)
        
        dK_sigma = m.exp(-theta[1]) * (K - m.exp(theta[2]) * np.eye(x.shape[0]))* L2norm
        dL1_sigma = -np.trace(K_inv.dot(dK_sigma))
        dL2_sigma = Kinvy.T.dot(dK_sigma).dot(Kinvy)
        
        dK_eta = m.exp(theta[2]) * np.eye(x.shape[0])
        dL1_eta = -np.trace(K_inv.dot(dK_eta))
        dL2_eta = Kinvy.T.dot(dK_eta).dot(Kinvy)
        
        dL_tau = dL1_tau + dL2_tau
        dL_sigma = dL1_sigma + dL2_sigma
        dL_eta = dL1_eta + dL2_eta
        
        return np.array([- dL_tau, - dL_sigma, - dL_eta])
    
    def fit(self, X, y):
        theta_ast, Lmin, result = fmin_l_bfgs_b(
            self.__logL, 
            self.theta_0,
            self.__dLdtheta,
            args=(X, y),
            maxiter=1000
        )
        
        return theta_ast, Lmin, result

In [15]:
GPR = GaussianProcessRegressionBFGS()
theta_ast, Lmin, result = GPR.fit(X_train, y_train)

[0. 0. 0.]
1.3562521447994338e+118
[[456.15776815]]
[-0.62706933  0.24869505 -0.73819701]
0.002103449437166388
[[343.81169683]]
[-1.04457892  1.19290883 -1.48807268]
3.821948941163889e-132
[[281.14816373]]
[-0.82344499  1.36263847 -1.16334067]
1.0460518165040945e-91
[[239.46820058]]
[-0.68289847  2.41588075 -0.87335579]
1.0569235693430669e-97
[[185.99028708]]
[-1.14882749  4.14976612  0.06554323]
3.934340204996515e+19
[[259.19654118]]
[-0.8003916   2.85311392 -0.63659401]
1.634678268855631e-78
[[173.84197317]]
[-0.79404537  3.41575492 -0.71052128]
8.56559233016334e-101
[[164.14173133]]
[-0.75668941  3.5931486  -0.73705672]
2.856170954012155e-107
[[163.35980682]]
[-0.71433067  3.68136728 -0.73513285]
1.0573606256360784e-107
[[163.10290521]]
[-0.53466291  3.86996672 -0.73276331]
6.588824829092957e-108
[[162.39226674]]
[-0.23493433  4.05695647 -0.72717576]
2.187287552096313e-106
[[161.61423378]]
[-0.03311234  4.15401021 -0.72200327]
8.62055661683766e-105
[[161.39405108]]
[ 0.03196766  4.1

In [16]:
result

{'grad': array([[[ 0.00035949]],
 
        [[-0.00026356]],
 
        [[-0.00066715]]]),
 'task': b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH',
 'funcalls': 19,
 'nit': 16,
 'warnflag': 0}

In [17]:
np.exp(theta_ast)

array([ 1.04629995, 66.40418798,  0.48637326])